In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
#Lendo o csv:
df = pd.read_csv("./Databases/ufc-master.csv")

#Filtrando os dados desejados:
df = df[["R_fighter","B_fighter","date","location","country","Winner","title_bout",
         "weight_class","gender","R_age","B_age","B_total_title_bouts","R_total_title_bouts"]]  

In [3]:
#Criando novas colunas no dataframe:
df['Winner_w'] = np.NaN
df['Loser'] = np.NaN
df['W_age'] = np.NaN
df['L_age'] = np.NaN
df['W_titles'] = np.NaN
df['L_titles'] = np.NaN

#Adicionando os vencedores na coluna Winner e os perdedores na coluna Loser,
#o mesmo com as idades nas respectivas colunas(W_age,L_age):
for index, row in df.iterrows():
    if row['Winner'] == "Blue":
        df.loc[index,'Winner_w'] =  df.loc[index,'B_fighter']
        df.loc[index,'Loser'] =  df.loc[index,'R_fighter']
        df.loc[index,'W_age'] =  df.loc[index,'B_age']
        df.loc[index,'L_age'] =  df.loc[index,'R_age']
        df.loc[index,'W_titles'] =  df.loc[index,'B_total_title_bouts']
        df.loc[index,'L_titles'] =  df.loc[index,'R_total_title_bouts']

    elif row['Winner'] == "Red":
        df.loc[index,'Winner_w'] =  df.loc[index,'R_fighter']
        df.loc[index,'Loser'] =  df.loc[index,'B_fighter']
        df.loc[index,'W_age'] =  df.loc[index,'R_age']
        df.loc[index,'L_age'] =  df.loc[index,'B_age']
        df.loc[index,'W_titles'] =  df.loc[index,'R_total_title_bouts']
        df.loc[index,'L_titles'] =  df.loc[index,'B_total_title_bouts']

In [4]:
#Filtrando novamente os dados, agora retirando as colunas R_fighter,B_fighter, Winner, R_age e B_age:
df = df[["Winner_w","Loser","date","location","country","title_bout",
         "weight_class","gender","W_age","L_age",'W_titles','L_titles']] 

#Renomeando a coluna Winner_w, chamando a de Winner:
df.rename(columns={'Winner_w':'Winner'}, inplace=True)

In [5]:
df["year"] = [i[-4:] for i in df["date"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Winner        4464 non-null   object 
 1   Loser         4464 non-null   object 
 2   date          4464 non-null   object 
 3   location      4464 non-null   object 
 4   country       4464 non-null   object 
 5   title_bout    4464 non-null   bool   
 6   weight_class  4464 non-null   object 
 7   gender        4464 non-null   object 
 8   W_age         4464 non-null   float64
 9   L_age         4464 non-null   float64
 10  W_titles      4464 non-null   float64
 11  L_titles      4464 non-null   float64
 12  year          4464 non-null   object 
dtypes: bool(1), float64(4), object(8)
memory usage: 423.0+ KB


In [31]:
df["title_bout"] = df["title_bout"].astype(int)#Fazendo essa mudança para não dar erro quando for fazer o dump para JSON
df.head(10)

,Winner,Loser,date,location,country,title_bout,weight_class,gender,W_age,L_age,W_titles,L_titles,year
0,Glover Teixeira,Thiago Santos,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Light Heavyweight,MALE,41.0,36.0,1.0,1.0,2020
1,Andrei Arlovski,Tanner Boser,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Heavyweight,MALE,41.0,29.0,5.0,0.0,2020
2,Raoni Barcelos,Khalid Taha,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Bantamweight,MALE,33.0,28.0,0.0,0.0,2020
3,Yan Xiaonan,Claudia Gadelha,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Women's Strawweight,FEMALE,31.0,31.0,0.0,1.0,2020
4,Trevin Giles,Bevon Lewis,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Middleweight,MALE,28.0,29.0,0.0,0.0,2020
5,Giga Chikadze,Jamey Simmons,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Featherweight,MALE,32.0,28.0,0.0,0.0,2020
6,Alexandr Romanov,Marcos Rogerio de Lima,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Heavyweight,MALE,29.0,35.0,0.0,0.0,2020
7,Darren Elkins,Eduardo Garagorri,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Featherweight,MALE,36.0,31.0,0.0,0.0,2020
8,Max Griffin,Ramiz Brahimaj,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Welterweight,MALE,34.0,27.0,0.0,0.0,2020
9,Gustavo Lopez,Anthony Birchak,11/7/2020,"Las Vegas, Nevada, USA",USA,0,Bantamweight,MALE,31.0,34.0,0.0,0.0,2020


## Representação
O banco de dados está representado da seguinte forma:  

![UFC_Schema](./Images/UFC_Schema.png)

## Tabela `fighters`

In [7]:
#Criando uma dataframe com todos os lutadores
df_fighters= df[["Winner","gender","W_age","W_titles",'year', 'date']]
df_fighters_l = df[["Loser","gender","L_age","L_titles",'year','date']]


df_fighters.columns = ['name', 'gender','age','title_bout','year','date']
df_fighters_l.columns = ['name', 'gender','age','title_bout','year','date']

df_fighters = df_fighters.append(df_fighters_l)


#Mudando os tipos de algumas colunas e ordenando pela data da luta

df_fighters["year"] = df_fighters["year"].astype(float)
df_fighters['date'] = pd.to_datetime(df_fighters['date'])
df_fighters["age"] = df_fighters["age"].astype(float)

df_fighters = df_fighters.sort_values(['date'], ascending = False)
df_fighters

,name,gender,age,title_bout,year,date
0,Glover Teixeira,MALE,41.0,1.0,2020.0,2020-11-07
1,Andrei Arlovski,MALE,41.0,5.0,2020.0,2020-11-07
9,Anthony Birchak,MALE,34.0,0.0,2020.0,2020-11-07
8,Ramiz Brahimaj,MALE,27.0,0.0,2020.0,2020-11-07
7,Eduardo Garagorri,MALE,31.0,0.0,2020.0,2020-11-07
...,...,...,...,...,...,...
4458,Vladimir Matyushenko,MALE,39.0,1.0,2010.0,2010-03-21
4457,Clay Guida,MALE,28.0,0.0,2010.0,2010-03-21
4453,Jon Jones,MALE,22.0,0.0,2010.0,2010-03-21
4454,Junior Dos Santos,MALE,26.0,0.0,2010.0,2010-03-21


In [8]:
#Criando um id para cada lutador
id_fighters_list = list(range(1,1651))

df_fighters['name'].unique()

id_fighters = dict(zip(df_fighters['name'].unique(),id_fighters_list))

#Adicionando a tabela id
df_fighters["id_fighters"] = [id_fighters[i] for i in df_fighters["name"]]

#Calculando o ano de nascimento e filtrando o dataframe
df_fighters["born_year"] = df_fighters["year"] - df_fighters["age"] 

df_fighters = df_fighters[['id_fighters','name', 'gender','title_bout',"born_year"]] 

#Pegando os dados mais recentes de cada lutador a partir do seu id
#Vale ressaltar  que há uma margem de erro no ano 

df_fighters  = df_fighters .drop_duplicates(subset='id_fighters', keep='first')
df_fighters.index = [i for i in range(len(df_fighters))]

In [9]:
df_fighters

,id_fighters,name,gender,title_bout,born_year
0,1,Glover Teixeira,MALE,1.0,1979.0
1,2,Andrei Arlovski,MALE,5.0,1979.0
2,3,Anthony Birchak,MALE,0.0,1986.0
3,4,Ramiz Brahimaj,MALE,0.0,1993.0
4,5,Eduardo Garagorri,MALE,0.0,1989.0
...,...,...,...,...,...
1645,1646,Nate Quarry,MALE,1.0,1972.0
1646,1647,Rory Markham,MALE,0.0,1982.0
1647,1648,Chase Gormley,MALE,0.0,1983.0
1648,1649,Shannon Gugerty,MALE,0.0,1982.0


### Criando uma função para fazer o dump desse dataframe

In [10]:
def dump_dataframe(nome_da_tabela, dataframe, tem_id = True, ajeitar_NULL = False):
    comeca = 0
    if tem_id:
        comeca = 1
    
    dataframe = dataframe[dataframe.columns[comeca:]]
        
    dump_dataframe = ""
    dump_dataframe += "INSERT INTO {} ".format(nome_da_tabela)
    dump_dataframe += "({})\n".format(", ".join(list(dataframe.columns)))
    dump_dataframe += "VALUES "
    for idx in dataframe.index:
        if idx == dataframe.index[-1]:
            dump_dataframe += "\t({});".format(list(dataframe.iloc[idx]))
        else:
            dump_dataframe += "\t({}),\n".format(list(dataframe.iloc[idx]))

    dump_dataframe = dump_dataframe.replace("[", '')
    dump_dataframe = dump_dataframe.replace("]", '')

    if ajeitar_NULL:
        import re
        dump_dataframe = re.sub(r"'(NULL)'", r"\1", dump_dataframe)
    
    file = open("./Dump/dump_UFC_Schema.sql", "a", encoding="UTF-8")
    file.write(dump_dataframe)
    file.write("\n\n")
    file.close()
    
    #return dump_dataframe

#### Fazendo o dump da tabela `fighters`

In [11]:
dump_dataframe("fighters", df_fighters)

## Tabela `year`

In [12]:
df_year = df['year']
df_year.columns = ['id_year']
df_year = df_year.drop_duplicates()
df_year.sort_values(inplace=True)
df_year.index = [i for i in range(1, len(df_year.index)+1)]

In [13]:
df_year

1     2010
2     2011
3     2012
4     2013
5     2014
6     2015
7     2016
8     2017
9     2018
10    2019
11    2020
Name: year, dtype: object

#### Fazendo o dump da tabela `year`

In [14]:
dump_year = ""
dump_year += "INSERT INTO year ({})\n".format(df_year.name)
dump_year += "VALUES "
for i in df_year:
    if df_year[len(df_year)] == i:
        dump_year += "\t({});".format(i)
    else:
        dump_year += "\t({}),\n".format(i)

dump_year = dump_year.replace("[", '')
dump_year = dump_year.replace("]", '')


file = open("./Dump/dump_UFC_Schema.sql", "a", encoding="UTF-8")
file.write(dump_year)
file.write("\n\n")
file.close()

## Tabela `weight_class`

In [15]:
df_weight_class = pd.DataFrame({"id_weight_class": [i for i in range(1, len(df["weight_class"].unique())+1)],
                     "name": df["weight_class"].unique()})
df_weight_class

,id_weight_class,name
0,1,Light Heavyweight
1,2,Heavyweight
2,3,Bantamweight
3,4,Women's Strawweight
4,5,Middleweight
5,6,Featherweight
6,7,Welterweight
7,8,Lightweight
8,9,Women's Flyweight
9,10,Catch Weight


#### Fazendo o dump da tabela `weight_class`

In [16]:
dump_dataframe("weight_class", df_weight_class)

## Tabela `location`

In [17]:
df.head()

,Winner,Loser,date,location,country,title_bout,weight_class,gender,W_age,L_age,W_titles,L_titles,year
0,Glover Teixeira,Thiago Santos,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Light Heavyweight,MALE,41.0,36.0,1.0,1.0,2020
1,Andrei Arlovski,Tanner Boser,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Heavyweight,MALE,41.0,29.0,5.0,0.0,2020
2,Raoni Barcelos,Khalid Taha,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Bantamweight,MALE,33.0,28.0,0.0,0.0,2020
3,Yan Xiaonan,Claudia Gadelha,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Women's Strawweight,FEMALE,31.0,31.0,0.0,1.0,2020
4,Trevin Giles,Bevon Lewis,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Middleweight,MALE,28.0,29.0,0.0,0.0,2020


`OBS.:` A maioria dos locais está dividido em três partes (city, state e country), porém, alguns não contém o estado, como é o caso de `Auckland` e `Montevideo` por exemplo.  
Então, foi colocado `NULL` no campo `state` desses locais.

In [18]:
uni_loc = df["location"].unique()
df_location = pd.DataFrame({"id_location": [i for i in range(1, len(uni_loc)+1)],
                           "city": [i.split(",")[0].strip() for i in uni_loc],
                           "state": [i.split(",")[1].strip() if len(i.split(",")) == 3 else "NULL" for i in uni_loc], 
                           "country": [i.split(",")[2].strip() if len(i.split(",")) == 3 else i.split(",")[1].strip() for i in uni_loc]})
df_location.head()

,id_location,city,state,country
0,1,Las Vegas,Nevada,USA
1,2,Abu Dhabi,Abu Dhabi,United Arab Emirates
2,3,Jacksonville,Florida,USA
3,4,Brasilia,Distrito Federal,Brazil
4,5,Norfolk,Virginia,USA


#### Fazendo o dump da tabela `location`

In [19]:
dump_dataframe("location", df_location, ajeitar_NULL = True)

## Tabela `fight`

In [20]:
df.head()

,Winner,Loser,date,location,country,title_bout,weight_class,gender,W_age,L_age,W_titles,L_titles,year
0,Glover Teixeira,Thiago Santos,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Light Heavyweight,MALE,41.0,36.0,1.0,1.0,2020
1,Andrei Arlovski,Tanner Boser,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Heavyweight,MALE,41.0,29.0,5.0,0.0,2020
2,Raoni Barcelos,Khalid Taha,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Bantamweight,MALE,33.0,28.0,0.0,0.0,2020
3,Yan Xiaonan,Claudia Gadelha,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Women's Strawweight,FEMALE,31.0,31.0,0.0,1.0,2020
4,Trevin Giles,Bevon Lewis,11/7/2020,"Las Vegas, Nevada, USA",USA,0.0,Middleweight,MALE,28.0,29.0,0.0,0.0,2020


Representação das colunas da tabela `fight`:
```python
columns = ["id_fight", "id_winner", "id_loser", "id_location", "id_year", "title_bout", "id_weight_class", "date"]
```

In [21]:
#Criando id_fight
id_fight = [i for i in range(1, len(df)+1)]

In [22]:
fighter_and_id = {df_fighters.loc[idx, "name"]: df_fighters.loc[idx, "id_fighters"] for idx in df_fighters.index}

id_winner = [fighter_and_id[name] for name in df["Winner"]]

id_loser = [fighter_and_id[name] for name in df["Loser"]]

In [25]:
location_and_id = {}
for idx in df_location.index:
    if df_location.loc[idx, "state"] != "NULL":
        location_and_id[df_location.loc[idx, "city"] + ", " + df_location.loc[idx, "state"] + ", " + df_location.loc[idx, "country"]] = df_location.loc[idx, "id_location"]
    else:
        location_and_id[df_location.loc[idx, "city"] + ", " + df_location.loc[idx, "country"]] = df_location.loc[idx, "id_location"]


In [26]:
id_location = [location_and_id[local] for local in df["location"]]

In [27]:
weight_class_and_id = {df_weight_class.loc[idx, "name"]: df_weight_class.loc[idx, "id_weight_class"] for idx in df_weight_class.index}

id_weight_class = [weight_class_and_id[w_class] for w_class in df["weight_class"]]

### Tudo pronto para criar a tabela `fight`

In [28]:
df_fight = pd.DataFrame({"id_fight": id_fight,
                        "id_winner": id_winner,
                        "id_loser": id_loser,
                        "id_location": id_location,
                        "id_year": df["year"],
                        "title_bout": df["title_bout"],
                        "if_weight_class": id_weight_class,
                        "date": df["date"]})

In [29]:
df_fight.head()

,id_fight,id_winner,id_loser,id_location,id_year,title_bout,if_weight_class,date
0,1,1,12,1,2020,0.0,1,11/7/2020
1,2,2,11,1,2020,0.0,2,11/7/2020
2,3,20,10,1,2020,0.0,3,11/7/2020
3,4,19,9,1,2020,0.0,4,11/7/2020
4,5,18,8,1,2020,0.0,5,11/7/2020


#### Fazendo o dump da tabela `fight`

In [30]:
dump_dataframe("fight", df_fight)